In [1]:
import pandas as pd 
import numpy as np
import pulp
from tabulate import tabulate
df = pd.read_excel("C:\\Users\\MHK\\Downloads\\Book2.xlsx")

In [2]:
df

,Region,Product,Price point,estimated quantity,Revenue,Original price,Gross Revenue,Cost of goods sold per unit,Total cost of goods,Total Margin
0,UAE,X,65,10,650,85,850,40,400,250
1,UAE,X,60,16,960,85,1360,40,640,320
2,UAE,X,55,15,825,85,1275,40,600,225
3,UAE,X,50,19,950,85,1615,40,760,190
4,UAE,X,45,23,1035,85,1955,40,920,115
5,UAE,X,40,23,920,85,1955,40,920,0
6,UAE,Y,80,5,400,80,400,50,250,150
7,UAE,Y,75,5,375,80,400,50,250,125
8,UAE,Y,70,9,630,80,720,50,450,180
9,UAE,Y,65,6,390,80,480,50,300,90


In [3]:
price_matrix = []
profit_matrix = []
revenue_matrix = []
grevenue_matrix = []
num_prices_per_product = []

for product in df['Product'].unique():
    
    price_points = sorted(df[df['Product'] == product]['Price point'].unique(), reverse=True)
    price_matrix.append(price_points)
    
    product_data = df[df['Product'] == product].pivot_table(
        index='Product', columns='Price point', values='Total Margin', aggfunc='sum', fill_value=0)
    profit_matrix.append(product_data.iloc[:, :].values.flatten()[::-1])

    product_data = df[df['Product'] == product].pivot_table(
        index='Product', columns='Price point', values='Revenue', aggfunc='sum', fill_value=0)
    revenue_matrix.append(product_data.iloc[:, :].values.flatten()[::-1])
    
    product_data = df[df['Product'] == product].pivot_table(
        index='Product', columns='Price point', values='Gross Revenue', aggfunc='sum', fill_value=0)
    grevenue_matrix.append(product_data.iloc[:, :].values.flatten()[::-1])

    num_prices_per_product.append(len(df[df['Product'] == product]['Price point'].unique()))

In [4]:
print("Profit Matrix:")
print(profit_matrix)
print("\nRevenue Matrix:")
print(revenue_matrix)
print("\nGross Revenue Matrix:")
print(grevenue_matrix)
print("\nNo of Prices per Product:")
print(num_prices_per_product)
print("\nPrice Points:")
print(price_matrix)

Profit Matrix:
[array([250, 320, 225, 190, 115,   0], dtype=int64), array([150, 125, 180,  90,  60,  30], dtype=int64), array([100, 100,  90,  70,  50,   0], dtype=int64), array([300, 350, 380, 315, 230, 120,  48], dtype=int64), array([420, 420, 380, 210], dtype=int64), array([ 900,  990, 1040, 1190, 1260, 1100, 1080,  840,  580,  435,  300],
      dtype=int64), array([434, 456, 468, 517, 588, 518, 576, 567, 484, 374], dtype=int64)]

Revenue Matrix:
[array([ 650,  960,  825,  950, 1035,  920], dtype=int64), array([400, 375, 630, 390, 360, 330], dtype=int64), array([300, 350, 390, 420, 550, 500], dtype=int64), array([500, 630, 760, 735, 690, 600, 528], dtype=int64), array([ 900,  980, 1140, 1050], dtype=int64), array([1800, 2090, 2340, 2890, 3360, 3300, 3780, 3640, 3480, 3335, 3300],
      dtype=int64), array([ 700,  760,  810,  935, 1120, 1050, 1260, 1365, 1320, 1210],
      dtype=int64)]

Gross Revenue Matrix:
[array([ 850, 1360, 1275, 1615, 1955, 1955], dtype=int64), array([400, 400,

In [44]:
problem = pulp.LpProblem("Maximize_Profits_With_Discount", pulp.LpMaximize)

# Decision variables
x = pulp.LpVariable.dicts("Price", ((product_name, j) for i, product_name in enumerate(df['Product'].unique()) for j in range(max(num_prices_per_product))), 0, 1, pulp.LpBinary)

# Objective function (maximize total profit)
total_profit = pulp.lpSum(profit_matrix[i][j] * x[(product_name, j)] for i, product_name in enumerate(df['Product'].unique()) for j in range(num_prices_per_product[i]))
problem += total_profit, "Total_Profit"

# Constraint: Each product must have exactly one selected price
for i, product_name in enumerate(df['Product'].unique()):
    problem += pulp.lpSum(x[(product_name, j)] for j in range(num_prices_per_product[i])) == 1, f"{product_name}_Price_Constraint"

# Discount Constraint
revenue_difference = pulp.lpSum((grevenue_matrix[i][j] - revenue_matrix[i][j]) * x[(product_name, j)] for i, product_name in enumerate(df['Product'].unique()) for j in range(num_prices_per_product[i]))
discount_constraint = 0.3 # Set your desired maximum discount
discount_tolerance = 0.005

# Divide revenue difference by the total gross revenue to obtain the selected discount percentage
total_gross_revenue = pulp.lpSum((grevenue_matrix[i][j]) * x[(product_name, j)] for i, product_name in enumerate(df['Product'].unique()) for j in range(num_prices_per_product[i]))
problem += revenue_difference  <= (discount_constraint + discount_tolerance) * total_gross_revenue, "Discount_Constraint_Upper"
problem += revenue_difference  >= (discount_constraint - discount_tolerance) * total_gross_revenue, "Discount_Constraint_Lower"

#Profit Constraint
profit_constraint = 2500
profit_tolerance = 0.05
problem += total_profit <= profit_constraint + (profit_tolerance * profit_constraint), "Profit_Constraint_Upper"
problem += total_profit >= profit_constraint - (profit_tolerance * profit_constraint), "Profit_Constraint_Lower"

# Solve the optimization problem
problem.solve()

# Print the results
print("Status:", pulp.LpStatus[problem.status])


# Extract and print the optimal prices
optimal_prices = np.zeros((len(df['Product'].unique()), max(num_prices_per_product)))
selected_prices = []
for i, product_name in enumerate(df['Product'].unique()):
    for j in range(max(num_prices_per_product)):
        optimal_prices[i][j] = pulp.value(x[(product_name, j)])
        if optimal_prices[i][j] == 1:
            # Find the actual value of the selected price point from the price_matrix
            price_point_index = j
            price_point_value = price_matrix[i][price_point_index]
            selected_prices.append((product_name, f"Price Point {price_point_index + 1}", price_point_value))

print("Optimal Prices:")
print(optimal_prices)

results_df = pd.DataFrame(columns=["Product", "Selected Price Point", "Value (AED)"])
# Populate the results DataFrame
for product, price_point, actual_value in selected_prices:
    results_df = results_df.append({"Product": product, "Selected Price Point": price_point, "Value (AED)": actual_value}, ignore_index=True)

# Print the results DataFrame
print(tabulate(results_df, headers='keys', tablefmt='fancy_grid', showindex=False))

print("Target Profit (AED) :", profit_constraint)
print("Total Profit (AED) :", pulp.value(problem.objective))

# Calculate the discount obtained by the selected price points
selected_discount = (pulp.value(revenue_difference) / pulp.value(total_gross_revenue)) * 100
print("Target Discount(%) :", discount_constraint * 100)
print("Discount (%) :", selected_discount)

Status: Optimal
Optimal Prices:
[[ 0.  1.  0.  0.  0.  0. nan nan nan nan nan]
 [ 0.  0.  0.  1.  0.  0. nan nan nan nan nan]
 [ 1.  0.  0.  0.  0.  0. nan nan nan nan nan]
 [ 0.  0.  0.  0.  0.  0.  1. nan nan nan nan]
 [ 1.  0.  0.  0. nan nan nan nan nan nan nan]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0. nan]]
╒═══════════╤════════════════════════╤═══════════════╕
│ Product   │ Selected Price Point   │   Value (AED) │
╞═══════════╪════════════════════════╪═══════════════╡
│ X         │ Price Point 2          │            60 │
├───────────┼────────────────────────┼───────────────┤
│ Y         │ Price Point 4          │            65 │
├───────────┼────────────────────────┼───────────────┤
│ Z         │ Price Point 1          │            75 │
├───────────┼────────────────────────┼───────────────┤
│ A         │ Price Point 7          │            22 │
├───────────┼────────────────────────┼───────────────┤
│ B         │ Price Point 1      